In [18]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_case as dr_c
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

In [19]:
importlib.reload(dt)
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dr_w)

anxiety_label_type = None
case_label_type = dr_c.SelfReports.AROUSAL
wesad_label_type = "stai"

threshold = "fixed"
test_size = 1.0
percent_of_target_dataset = 0.0

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
# ]
] + train.Metrics.STATISTICAL

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]
model_phases_wesad = dr_w.Phases.PHASE_ORDER

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

# temp_a, _ = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=False)
# idx = temp_a[temp_a["bpm"] > 200].index 
# invalid_case_subjects = set(temp_a["subject"].iloc[idx].tolist())
# idx = temp_a[temp_a["bpm"] < 35].index 
# invalid_case_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

# temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
# idx = temp_a[temp_a["bpm"] > 200].index 
# invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
# idx = temp_a[temp_a["bpm"] < 35].index 
# invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# x_c, y_c = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
# x_w, y_w = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
# x_c = x_c.drop(["phaseId"], axis=1)
# x_w = x_w.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
# inds = pd.isnull(x_c).any(axis=1).to_numpy().nonzero()[0]
# x_c = x_c.drop(labels=inds, axis=0).reset_index(drop=True)
# y_c = y_c.drop(labels=inds, axis=0).reset_index(drop=True)
# inds = pd.isnull(x_w).any(axis=1).to_numpy().nonzero()[0]
# x_w = x_w.drop(labels=inds, axis=0).reset_index(drop=True)
# y_w = y_w.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
# x_c["subject"] = x_c["subject"] + 500
# y_c["subject"] = y_c["subject"] + 500
# x_w["subject"] = x_w["subject"] + 1000
# y_w["subject"] = y_w["subject"] + 1000

In [20]:
importlib.reload(dt)
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dr_w)

import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM


endog = y_a.loc[:, "label"]
exog = x_a.loc[:, x_a.columns != "subject"]
groups = x_a.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("APD " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

######################################################################################

endog = y_c.loc[:, "label"]
exog = x_c.loc[:, x_c.columns != "subject"]
groups = x_c.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("CASE " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

######################################################################################

endog = y_w.loc[:, "label"]
exog = x_w.loc[:, x_w.columns != "subject"]
groups = x_w.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("WESAD " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

APD -------------------------------------------------

            Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   label    
No. Observations:   1032      Method:               REML     
No. Groups:         52        Scale:                0.1661   
Min. group size:    18        Log-Likelihood:       -582.3693
Max. group size:    29        Converged:            Yes      
Mean group size:    19.8                                     
-------------------------------------------------------------
             Coef.   Std.Err.   z    P>|z|   [0.025   0.975] 
-------------------------------------------------------------
bpm            0.781    0.148  5.265 0.000     0.490    1.072
rmssd         -0.153    0.267 -0.574 0.566    -0.677    0.370
hf_rr         10.325   20.390  0.506 0.613   -29.639   50.290
lf_rr        -35.994   77.549 -0.464 0.643  -187.987  115.999
sdnn           0.269    0.260  1.033 0.302    -0.241    0.779
mean_SCL       6.185    5.5

Maximum Likelihood optimization failed to converge. Check mle_retvals
Retrying MixedLM optimization with lbfgs
Maximum Likelihood optimization failed to converge. Check mle_retvals
Retrying MixedLM optimization with cg
Maximum Likelihood optimization failed to converge. Check mle_retvals
MixedLM optimization failed, trying a different optimizer may help.
Gradient optimization failed, |grad| = 10.461974


CASE -------------------------------------------------

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: label    
No. Observations: 480     Method:             REML     
No. Groups:       30      Scale:              0.1206   
Min. group size:  16      Log-Likelihood:     -197.9019
Max. group size:  16      Converged:          No       
Mean group size:  16.0                                 
-------------------------------------------------------
            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-------------------------------------------------------
bpm          0.982    0.205  4.782 0.000   0.579  1.384
rmssd        0.263    0.243  1.086 0.278  -0.212  0.739
hf_rr        2.389    0.974  2.454 0.014   0.481  4.297
lf_rr       -2.321    0.845 -2.748 0.006  -3.976 -0.666
sdnn        -0.142    0.226 -0.630 0.529  -0.585  0.300
mean_SCL    -0.147    0.123 -1.199 0.230  -0.388  0.093
SCR_rate     0.120    0.181  0.665 0.506  -0.235  0.476
e